# SModelS – a tool for interpreting simplified-model results from the LHC

 <img src=https://smodels.github.io/images/banner720.png />

## WW, Nov 2025: This tutorial is outdated--
## See https://smodels.readthedocs.io/en/stable/Examples.html for up-to-date entry points to SModelS!

Jan Heisig and Andre Lessa, for the SModelS collaboration.

Re-Interpretation Workshop 2021, February 2021. Updated for SModelS 2.3.2 and google colab by WW, Sept 2023.

## Preparatory step -- installation:

### Prepare the environment: install smodels, download files needed for the tutorial

In [1]:
!rm -f prepare.sh ; wget https://raw.githubusercontent.com/SModelS/tutorials/main/prepare.sh ; sh ./prepare.sh

--2023-09-12 17:38:39--  https://raw.githubusercontent.com/SModelS/tutorials/main/prepare.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230 [text/plain]
Saving to: 'prepare.sh'

prepare.sh          100%[===================>]     230  --.-KB/s    in 0s      

2023-09-12 17:38:39 (3.70 MB/s) - 'prepare.sh' saved [230/230]

--2023-09-12 17:38:42--  https://raw.githubusercontent.com/SModelS/tutorials/main/files_tutorial.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7195873 (6.9M) [application/octet-stream]
Saving to: 'fi

### Check installation:

In [2]:
!smodelsTools.py installation

                   ____  __  __           _      _ ____
                  / ___||  \/  | ___   __| | ___| / ___|
                  \___ \| |\/| |/ _ \ / _` |/ _ \ \___ \
                   ___) | |  | | (_) | (_| |  __/ |___) |
                  |____/|_|  |_|\___/ \__,_|\___|_|____/


SModelS -- A tool for interpreting simplified-model results from the LHC,
see https://smodels.github.io/.

Copyright (C) 2012-2023 The SModelS collaboration, smodels-users@lists.oeaw.ac.at
Current members: Mohammad Mahdi Altakach, Sabine Kraml, Andre Lessa, Sahana Narashima, Timothee Pascal, Humberto Reyes-Gonzalez, Wolfgang Waltenberger

Previously involved in SModelS: Gael Alguero, Federico Ambrogi, Juhi Dutta, Jan Heisig, Charanjit K. Khosa, Suchita Kulkarni, Ursula Laa, Veronika Magerl, Wolfgang Magerl, Philipp Neuhuber, Doris Proschofsky, Jory Sonneveld, Michael Traub, Matthias Wolf, Alicia Wongel

SModelS version: 2.3.2
Installation directory: /home/walten/git/smodels/
This binary: /home/walten/git

## First Step: Run SModelS for a single file

In [ ]:
!runSModelS.py -p parameters.ini -f slhaFiles/wino_11010599.slha

INFO in databaseObj.loadBinaryFile() in 505: loading binary db file /home/walten/.cache/smodels/official230.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 512: Loaded database from /home/walten/.cache/smodels/official230.pcl in 1.5 secs.
INFO in modelTester.testPoints() in 341: Running SModelS for a single file
WARNING in printer.setOutPutFile() in 841: Removing old output file ./results/wino_11010599.slha.py
WARNING in printer.setOutPutFile() in 670: Removing old output file ./results/wino_11010599.slha.smodels
INFO in model.updateParticles() in 394: Loaded 62 BSM particles
INFO in model.updateParticles() in 394: Loaded 62 BSM particles
Input status: 1
Decomposition output status: 1 #decomposition was successful
# Input File: slhaFiles/wino_11010599.slha
# checkinput = True
# combinesrs = False
# computestatistics = True
# docompress = True
# doinvisible = True
# maxcond = 0.2
# minmassgap = 5.
# model = share.models.mssm
# ncpus = 1
# promptwidth = 1e-8
# sigmacut = 0

## Second Step: Run SModelS for multiple files

In [ ]:
!runSModelS.py -p parameters_idm.ini -f slhaFiles/IDM_points -o results/IDM_output

## Third Step: plot the results

### Plot IDM results

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import pyslha
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import glob,os,importlib
cm = matplotlib.colormaps.get_cmap('RdYlBu')

### Define path to SLHA and results folders

In [ ]:
slhaFolder = './slhaFiles/IDM_points'
resultsFolder = './results/IDM_output'

### Read SModelS results

In [ ]:
#Convert Experimental Results list to a dictionary
data = []
for f in glob.glob(resultsFolder+'/*.py'):
    spec = importlib.util.spec_from_file_location("output", f)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    smodelsDict = module.smodelsOutput
    slhaFile = os.path.basename(smodelsDict['OutputStatus']['input file'])
    slhaFile = os.path.join(slhaFolder,slhaFile)
    #Read SLHA file (using pyslha):
    slhaData = pyslha.readSLHAFile(slhaFile)
    data.append((slhaData,smodelsDict))

In [ ]:
mHc = []
ctau = []
r = []
for slhaData,smodelsDict in data:
    if not 'ExptRes' in smodelsDict: #If no results were applicable, point is allowed
        rmax = 0.0
    else:
        rmax = smodelsDict['ExptRes'][0]['r'] #First result is the most constraining
    mass = slhaData.blocks['MASS'][37] #H+ mass
    width = slhaData.decays[37].totalwidth #H+ mass
    if width:    
        ct = 1.967e-16/width
    else:
        ct = 1e7
    
    mHc.append(mass)
    ctau.append(ct)
    r.append(rmax)

### Plot results:

In [ ]:
plt.figure(figsize=(8,6))
cp = plt.scatter(mHc,ctau, c=r, vmin=0., vmax=2., s=25, cmap=cm)
cb = plt.colorbar(cp)
plt.yscale('log')
plt.xlabel(r'$m_{H^{\pm}}$ (GeV)',fontsize=20)    
plt.ylabel(r'$c\tau$ (m)',fontsize=20)
cb.set_label(r'$r=\sigma_{th}/\sigma_{UL}$',fontsize=20)
plt.tight_layout()
plt.show()